In [1]:
from mlaut.analyze_results import AnalyseResults
from mlaut.data import Data
import pandas as pd
from mlaut.estimators.estimators import instantiate_default_estimators
from mlaut.analyze_results.scores import ScoreAccuracy

import matplotlib.pyplot as plt
pd.options.display.max_rows = 1000

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data = Data()
input_io = data.open_hdf5('data/openml.h5', mode='r')
out_io = data.open_hdf5('data/openml-classification.h5', mode='a')
analyze = AnalyseResults(hdf5_output_io=out_io, 
                        hdf5_input_io=input_io, 
                        input_h5_original_datasets_group='openml/', 
                        output_h5_predictions_group='experiments/predictions/')


### All datasets

In [3]:
estimators = instantiate_default_estimators(['Classification'])
score_accuracy = ScoreAccuracy()

# (errors_per_estimator, 
#  errors_per_dataset_per_estimator) = analyze.prediction_errors(metric=score_accuracy, estimators=estimators)
 
(errors_per_estimator, 
 errors_per_dataset_per_estimator, 
 errors_per_dataset_per_estimator_df) = analyze.prediction_errors(score_accuracy, estimators)

2018-06-10 14:24:44,830 [MainThread  ] [WARNI]  Predictions for estimator SVC unavailable for dataset: GesturePhaseSegmentationProcessed. Analyse Results will skip GesturePhaseSegmentationProcessed.
2018-06-10 14:24:44,830 [MainThread  ] [WARNI]  Predictions for estimator SVC unavailable for dataset: GesturePhaseSegmentationProcessed. Analyse Results will skip GesturePhaseSegmentationProcessed.
2018-06-10 14:24:44,832 [MainThread  ] [WARNI]  Predictions for estimator GaussianNaiveBayes unavailable for dataset: GesturePhaseSegmentationProcessed. Analyse Results will skip GesturePhaseSegmentationProcessed.
2018-06-10 14:24:44,832 [MainThread  ] [WARNI]  Predictions for estimator GaussianNaiveBayes unavailable for dataset: GesturePhaseSegmentationProcessed. Analyse Results will skip GesturePhaseSegmentationProcessed.
2018-06-10 14:24:44,836 [MainThread  ] [WARNI]  Predictions for estimator BernoulliNaiveBayes unavailable for dataset: GesturePhaseSegmentationProcessed. Analyse Results will

#### Simple average and standard error

In [4]:
avg_and_std_error = analyze.average_and_std_error(errors_per_estimator)
# avg_and_std_error.index.name='Estimator Name'
avg_and_std_error

,avg_score,std_error
BaselineClassifier,0.560825,0.020387
NeuralNetworkDeepClassifier,0.601127,0.024979
BernoulliNaiveBayes,0.657803,0.017503
SVC,0.675144,0.018987
K_Neighbours,0.838803,0.022794
GaussianNaiveBayes,0.843921,0.026747
PassiveAggressiveClassifier,0.856268,0.025228
GradientBoostingClassifier,0.860161,0.019453
RandomForestClassifier,0.885413,0.018639
BaggingClassifier,0.888241,0.019002


#### Average Rank

In [5]:
avg_rank = analyze.ranks(errors_per_estimator, ascending=False)
avg_rank

,avg_rank
BaggingClassifier,2.68
RandomForestClassifier,2.87
PassiveAggressiveClassifier,3.59
GaussianNaiveBayes,4.50
GradientBoostingClassifier,4.73
K_Neighbours,5.07
SVC,6.87
BernoulliNaiveBayes,7.35
NeuralNetworkDeepClassifier,8.15
BaselineClassifier,9.19


#### Training time

In [18]:
avg_training_time, trainig_time_per_dataset = analyze.average_training_time(estimators)
avg_training_time

,avg training time (in sec)
BaggingClassifier,31.251519
BaselineClassifier,0.000646
BernoulliNaiveBayes,0.025111
GaussianNaiveBayes,0.013008
GradientBoostingClassifier,48.118384
K_Neighbours,41.033470
NeuralNetworkDeepClassifier,1.781616
PassiveAggressiveClassifier,38.951718
RandomForestClassifier,43.989733
SVC,684.234771


#### merge avg score, rank and training time

In [7]:
avg_metrics = pd.DataFrame.merge(avg_rank,avg_and_std_error, left_index=True, right_index=True)
avg_metrics = pd.DataFrame.merge(avg_metrics, avg_training_time,left_index=True, right_index=True)
avg_metrics


,avg_rank,avg_score,std_error,avg training time (in sec)
BaggingClassifier,2.68,0.888241,0.019002,31.251519
RandomForestClassifier,2.87,0.885413,0.018639,43.989733
PassiveAggressiveClassifier,3.59,0.856268,0.025228,38.951718
GaussianNaiveBayes,4.50,0.843921,0.026747,0.013008
GradientBoostingClassifier,4.73,0.860161,0.019453,48.118384
K_Neighbours,5.07,0.838803,0.022794,41.033470
SVC,6.87,0.675144,0.018987,684.234771
BernoulliNaiveBayes,7.35,0.657803,0.017503,0.025111
NeuralNetworkDeepClassifier,8.15,0.601127,0.024979,1.781616
BaselineClassifier,9.19,0.560825,0.020387,0.000646


#### Cohen's d

In [8]:
cohens_d = analyze.cohens_d(errors_per_estimator)
cohens_d

estimator_2,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,K_Neighbours,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
estimator_1,,,,,,,,,
BaggingClassifier,-2.34966,-1.78396,-0.270165,-0.206515,-0.333192,-1.82963,-0.202466,-0.0212487,-1.586587
BaselineClassifier,,0.721849,1.68358,2.12456,1.818,0.249991,1.82171,2.3501,0.820695
BernoulliNaiveBayes,,,1.16453,1.54662,1.25964,-0.371642,1.29272,1.78036,0.134305
GaussianNaiveBayes,,,,0.0982105,-0.0291291,-1.32686,0.0671604,0.254545,-1.029104
GradientBoostingClassifier,,,,,-0.142552,-1.63635,-0.0244456,0.187453,-1.361264
K_Neighbours,,,,,,-1.40572,0.102734,0.316595,-1.103349
NeuralNetworkDeepClassifier,,,,,,,1.43733,1.82429,0.471811
PassiveAggressiveClassifier,,,,,,,,0.185834,-1.147268
RandomForestClassifier,,,,,,,,,-1.580547


#### t-test

In [9]:
t_test, t_test_df = analyze.t_test(errors_per_estimator)
t_test_df

BaggingClassifier                \
                                       t_stat         p_val   
BaggingClassifier                    0.000000  1.000000e+00   
BaselineClassifier                 -11.630227  3.682695e-20   
BernoulliNaiveBayes                 -8.830166  4.169026e-14   
GaussianNaiveBayes                  -1.337248  1.842372e-01   
GradientBoostingClassifier          -1.022195  3.092054e-01   
K_Neighbours                        -1.649216  1.023054e-01   
NeuralNetworkDeepClassifier         -9.056194  1.351684e-14   
PassiveAggressiveClassifier         -1.002157  3.187361e-01   
RandomForestClassifier              -0.105176  9.164513e-01   
SVC                                 -7.853203  5.210719e-12   

                            BaselineClassifier                \
                                        t_stat         p_val   
BaggingClassifier                    11.630227  3.682695e-20   
BaselineClassifier                    0.000000  1.000000e+00   
BernoulliNaiveBayes                   3.572969  5.494195e-04   
GaussianNaiveBayes                    8.333276  4.908401e-13   
GradientBoostingClassifier           10.516055  9.208801e-18   
K_Neighbours                          8.998659  1.800852e-14   
NeuralNetworkDeepClassifier           1.237390  2.188989e-01   
PassiveAggressiveClassifier           9.017018  1.643330e-14   
RandomForestClassifier               11.632411  3.643322e-20   
SVC                                   4.062234  9.802301e-05   

                            BernoulliNaiveBayes                \
                                         t_stat         p_val   
BaggingClassifier                      8.830166  4.169026e-14   
BaselineClassifier                    -3.572969  5.494195e-04   
BernoulliNaiveBayes                    0.000000  1.000000e+00   
GaussianNaiveBayes                     5.764150  9.521242e-08   
GradientBoostingClassifier             7.655367  1.368733e-11   
K_Neighbours                           6.234898  1.144367e-08   
NeuralNetworkDeepClassifier           -1.839537  6.886362e-02   
PassiveAggressiveClassifier            6.398630  5.393426e-09   
RandomForestClassifier                 8.812353  4.555556e-14   
SVC                                    0.664776  5.077554e-01   

                            GaussianNaiveBayes                \
                                        t_stat         p_val   
BaggingClassifier                     1.337248  1.842372e-01   
BaselineClassifier                   -8.333276  4.908401e-13   
BernoulliNaiveBayes                  -5.764150  9.521242e-08   
GaussianNaiveBayes                    0.000000  1.000000e+00   
GradientBoostingClassifier            0.486117  6.279695e-01   
K_Neighbours                         -0.144182  8.856531e-01   
NeuralNetworkDeepClassifier          -6.567637  2.462658e-09   
PassiveAggressiveClassifier           0.332427  7.402768e-01   
RandomForestClassifier                1.259932  2.106860e-01   
SVC                                  -5.093807  1.703028e-06   

                            GradientBoostingClassifier                \
                                                t_stat         p_val   
BaggingClassifier                             1.022195  3.092054e-01   
BaselineClassifier                          -10.516055  9.208801e-18   
BernoulliNaiveBayes                          -7.655367  1.368733e-11   
GaussianNaiveBayes                           -0.486117  6.279695e-01   
GradientBoostingClassifier                    0.000000  1.000000e+00   
K_Neighbours                                 -0.705595  4.821141e-01   
NeuralNetworkDeepClassifier                  -8.099522  1.555219e-12   
PassiveAggressiveClassifier                  -0.121000  9.039390e-01   
RandomForestClassifier                        0.927847  3.557665e-01   
SVC                                          -6.737912  1.109920e-09   

                            K_Neighbours                \
                             

#### sign test

In [10]:
sign_test, sign_test_df = analyze.sign_test(errors_per_estimator)
sign_test_df

BaggingClassifier                \
                                       t_stat         p_val   
BaggingClassifier                    0.000000  1.000000e+00   
BaselineClassifier                  -7.517711  5.574373e-14   
BernoulliNaiveBayes                 -6.690452  2.224820e-11   
GaussianNaiveBayes                  -1.140927  2.539002e-01   
GradientBoostingClassifier          -1.733796  8.295434e-02   
K_Neighbours                        -1.847544  6.466838e-02   
NeuralNetworkDeepClassifier         -7.004121  2.485413e-12   
PassiveAggressiveClassifier         -0.596315  5.509645e-01   
RandomForestClassifier              -0.365372  7.148334e-01   
SVC                                 -6.449168  1.124655e-10   

                            BaselineClassifier                \
                                        t_stat         p_val   
BaggingClassifier                     7.517711  5.574373e-14   
BaselineClassifier                    0.000000  1.000000e+00   
BernoulliNaiveBayes                   3.157370  1.591995e-03   
GaussianNaiveBayes                    6.425040  1.318347e-10   
GradientBoostingClassifier            7.307449  2.722615e-13   
K_Neighbours                          6.593939  4.283086e-11   
NeuralNetworkDeepClassifier           1.499406  1.337684e-01   
PassiveAggressiveClassifier           6.593939  4.283086e-11   
RandomForestClassifier                7.569414  3.749109e-14   
SVC                                   3.905349  9.408951e-05   

                            BernoulliNaiveBayes                \
                                         t_stat         p_val   
BaggingClassifier                      6.690452  2.224820e-11   
BaselineClassifier                    -3.157370  1.591995e-03   
BernoulliNaiveBayes                    0.000000  1.000000e+00   
GaussianNaiveBayes                     5.446118  5.148108e-08   
GradientBoostingClassifier             6.218226  5.028085e-10   
K_Neighbours                           5.528844  3.223487e-08   
NeuralNetworkDeepClassifier           -1.375317  1.690332e-01   
PassiveAggressiveClassifier            5.773574  7.760743e-09   
RandomForestClassifier                 6.704240  2.024575e-11   
SVC                                    0.782449  4.339510e-01   

                            GaussianNaiveBayes                \
                                        t_stat         p_val   
BaggingClassifier                     1.140927  2.539002e-01   
BaselineClassifier                   -6.425040  1.318347e-10   
BernoulliNaiveBayes                  -5.446118  5.148108e-08   
GaussianNaiveBayes                    0.000000  1.000000e+00   
GradientBoostingClassifier           -0.592869  5.532692e-01   
K_Neighbours                         -0.854834  3.926432e-01   
NeuralNetworkDeepClassifier          -5.908004  3.462784e-09   
PassiveAggressiveClassifier           0.437758  6.615620e-01   
RandomForestClassifier                0.820365  4.120083e-01   
SVC                                  -5.191046  2.091155e-07   

                            GradientBoostingClassifier                \
                                                t_stat         p_val   
BaggingClassifier                             1.733796  8.295434e-02   
BaselineClassifier                           -7.307449  2.722615e-13   
BernoulliNaiveBayes                          -6.218226  5.028085e-10   
GaussianNaiveBayes                            0.592869  5.532692e-01   
GradientBoostingClassifier                    0.000000  1.000000e+00   
K_Neighbours                                 -0.465333  6.416931e-01   
NeuralNetworkDeepClassifier                  -6.652536  2.880850e-11   
PassiveAggressiveClassifier                   0.885856  3.756952e-01   
RandomForestClassifier                        1.458043  1.448287e-01   
SVC                                          -5.880428  4.092061e-09   

                            K_Neighbours                \
                             

#### t-test with Bonferroni correction

In [11]:
t_test_bonferroni_df = analyze.t_test_with_bonferroni_correction(errors_per_estimator)
t_test_bonferroni_df

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,K_Neighbours,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
BaggingClassifier,False,True,True,False,False,False,True,False,False,True
BaselineClassifier,True,False,False,True,True,True,False,True,True,True
BernoulliNaiveBayes,True,False,False,True,True,True,False,True,True,False
GaussianNaiveBayes,False,True,True,False,False,False,True,False,False,True
GradientBoostingClassifier,False,True,True,False,False,False,True,False,False,True
K_Neighbours,False,True,True,False,False,False,True,False,False,True
NeuralNetworkDeepClassifier,True,False,False,True,True,True,False,True,True,False
PassiveAggressiveClassifier,False,True,True,False,False,False,True,False,False,True
RandomForestClassifier,False,True,True,False,False,False,True,False,False,True
SVC,True,True,False,True,True,True,False,True,True,False


#### Wilcoxon test

In [12]:
_, wilcoxon_df_multiindex = analyze.wilcoxon_test(errors_per_estimator)
wilcoxon_df_multiindex

/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/scipy/stats/morestats.py:2385: UserWarning: Warning: sample size too small for normal approximation.
  warnings.warn("Warning: sample size too small for normal approximation.")
/media/viktor/Data/PhD/mlaut/py36/local/lib/python3.6/site-packages/scipy/stats/morestats.py:2410: RuntimeWarning: invalid value encountered in double_scalars
  z = (T - mn - correction) / se


BaggingClassifier                \
                                    statistic         p_val   
BaggingClassifier                         0.0           NaN   
BaselineClassifier                        1.0  8.029281e-10   
BernoulliNaiveBayes                      10.0  2.056477e-09   
GaussianNaiveBayes                      144.0  4.199190e-05   
GradientBoostingClassifier               12.0  3.897951e-08   
K_Neighbours                             23.0  8.423214e-08   
NeuralNetworkDeepClassifier               3.0  1.972217e-09   
PassiveAggressiveClassifier             222.5  1.941369e-02   
RandomForestClassifier                  287.0  4.698520e-01   
SVC                                      16.0  4.444366e-09   

                            BaselineClassifier                \
                                     statistic         p_val   
BaggingClassifier                          1.0  8.029281e-10   
BaselineClassifier                         0.0           NaN   
BernoulliNaiveBayes                       86.0  1.629863e-07   
GaussianNaiveBayes                        24.0  3.174959e-09   
GradientBoostingClassifier                 1.0  1.181233e-09   
K_Neighbours                               8.0  1.226241e-09   
NeuralNetworkDeepClassifier              313.0  2.889708e-03   
PassiveAggressiveClassifier               32.5  5.210640e-09   
RandomForestClassifier                     0.0  7.556929e-10   
SVC                                       68.0  3.851212e-08   

                            BernoulliNaiveBayes                \
                                      statistic         p_val   
BaggingClassifier                          10.0  2.056477e-09   
BaselineClassifier                         86.0  1.629863e-07   
BernoulliNaiveBayes                         0.0           NaN   
GaussianNaiveBayes                        115.5  4.678707e-07   
GradientBoostingClassifier                 24.0  4.799380e-09   
K_Neighbours                               59.0  9.090374e-08   
NeuralNetworkDeepClassifier               103.0  2.277773e-02   
PassiveAggressiveClassifier                82.0  1.312874e-07   
RandomForestClassifier                      3.0  1.337087e-09   
SVC                                        24.0  1.329573e-01   

                            GaussianNaiveBayes                \
                                     statistic         p_val   
BaggingClassifier                        144.0  4.199190e-05   
BaselineClassifier                        24.0  3.174959e-09   
BernoulliNaiveBayes                      115.5  4.678707e-07   
GaussianNaiveBayes                         0.0           NaN   
GradientBoostingClassifier               574.0  8.858215e-01   
K_Neighbours                             440.0  5.209637e-01   
NeuralNetworkDeepClassifier               49.5  1.377512e-08   
PassiveAggressiveClassifier              331.5  3.577126e-02   
RandomForestClassifier                   221.5  8.343457e-04   
SVC                                      172.0  7.002871e-06   

                            GradientBoostingClassifier                \
                                             statistic         p_val   
BaggingClassifier                                 12.0  3.897951e-08   
BaselineClassifier                                 1.0  1.181233e-09   
BernoulliNaiveBayes                               24.0  4.799380e-09   
GaussianNaiveBayes                               574.0  8.858215e-01   
GradientBoostingClassifier                         0.0           NaN   
K_Neighbours                                     333.5  2.372403e-02   
NeuralNetworkDeepClassifier                       13.0  1.654806e-09   
PassiveAggressiveClassifier                      326.5  1.180589e-01   
RandomForestClassifier                            80.0  3.398935e-06   
SVC                                               70.0  6.798850e-08   

                            K_Neighbours                \
                             

#### Friedman test

In [13]:
friedman_test, friedman_test_df = analyze.friedman_test(errors_per_estimator)
friedman_test_df

,statistic,p_value
0,261.467136,3.771694e-51


In [14]:
nemeniy_test = analyze.nemenyi(errors_per_estimator)
nemeniy_test_df = pd.DataFrame(nemeniy_test)
nemeniy_test_df

,BaggingClassifier,BaselineClassifier,BernoulliNaiveBayes,GaussianNaiveBayes,GradientBoostingClassifier,K_Neighbours,NeuralNetworkDeepClassifier,PassiveAggressiveClassifier,RandomForestClassifier,SVC
BaggingClassifier,-1.000000e+00,9.770686e-12,0.000002,9.984100e-01,9.982158e-01,9.782782e-01,1.067945e-08,9.999413e-01,1.000000e+00,0.000018
BaselineClassifier,9.770686e-12,-1.000000e+00,0.953511,2.781690e-08,3.091460e-08,4.985285e-07,9.994875e-01,2.384100e-09,3.629285e-11,0.821255
BernoulliNaiveBayes,1.503125e-06,9.535112e-01,-1.000000,4.245506e-04,4.559163e-04,2.843343e-03,9.998985e-01,7.837964e-05,3.934614e-06,0.999999
GaussianNaiveBayes,9.984100e-01,2.781690e-08,0.000425,-1.000000e+00,1.000000e+00,9.999991e-01,8.684415e-06,9.999999e-01,9.995883e-01,0.002767
GradientBoostingClassifier,9.982158e-01,3.091460e-08,0.000456,1.000000e+00,-1.000000e+00,9.999993e-01,9.473927e-06,9.999998e-01,9.995259e-01,0.002944
K_Neighbours,9.782782e-01,4.985285e-07,0.002843,9.999991e-01,9.999993e-01,-1.000000e+00,9.138695e-05,9.998588e-01,9.904547e-01,0.014219
NeuralNetworkDeepClassifier,1.067945e-08,9.994875e-01,0.999899,8.684415e-06,9.473927e-06,9.138695e-05,-1.000000e+00,1.127913e-06,3.280185e-08,0.995981
PassiveAggressiveClassifier,9.999413e-01,2.384100e-09,0.000078,9.999999e-01,9.999998e-01,9.998588e-01,1.127913e-06,-1.000000e+00,9.999937e-01,0.000628
RandomForestClassifier,1.000000e+00,3.629285e-11,0.000004,9.995883e-01,9.995259e-01,9.904547e-01,3.280185e-08,9.999937e-01,-1.000000e+00,0.000043
SVC,1.817120e-05,8.212545e-01,0.999999,2.767068e-03,2.944233e-03,1.421924e-02,9.959812e-01,6.276147e-04,4.333338e-05,-1.000000


### Per dataset

In [15]:
errors_per_dataset_per_estimator_df

loss  \
AP_Breast_Colon                                    BaggingClassifier            0.048077   
                                                   BaselineClassifier           0.514423   
                                                   BernoulliNaiveBayes          0.471154   
                                                   GaussianNaiveBayes           0.072115   
                                                   GradientBoostingClassifier   0.057692   
                                                   K_Neighbours                 0.086538   
                                                   NeuralNetworkDeepClassifier  0.528846   
                                                   PassiveAggressiveClassifier  0.057692   
                                                   RandomForestClassifier       0.057692   
                                                   SVC                          0.471154   
AP_Breast_Kidney                                   BaggingClassifier            0.020000   
                                                   BaselineClassifier           0.465000   
                                                   BernoulliNaiveBayes          0.410000   
                                                   GaussianNaiveBayes           0.010000   
                                                   GradientBoostingClassifier   0.035000   
                                                   K_Neighbours                 0.045000   
                                                   NeuralNetworkDeepClassifier  0.410000   
                                                   PassiveAggressiveClassifier  0.045000   
                                                   RandomForestClassifier       0.025000   
                                                   SVC                          0.410000   
AP_Breast_Lung                                     BaggingClassifier            0.025641   
                                                   BaselineClassifier           0.371795   
                                                   BernoulliNaiveBayes          0.282051   
                                                   GaussianNaiveBayes           0.038462   
                                                   GradientBoostingClassifier   0.025641   
                                                   K_Neighbours                 0.083333   
                                                   NeuralNetworkDeepClassifier  0.282051   
                                                   PassiveAggressiveClassifier  0.038462   
                                                   RandomForestClassifier       0.025641   
                                                   SVC                          0.282051   
AP_Breast_Omentum                                  BaggingClassifier            0.064748   
                                                   BaselineClassifier           0.323741   
                                                   BernoulliNaiveBayes          0.215827   
                                                   GaussianNaiveBayes           0.050360   
                                                   GradientBoostingClassifier   0.057554   
                                                   K_Neighbours                 0.093525   
                                                   NeuralNetworkDeepClassifier  0.597122   
                                                   PassiveAggressiveClassifier  0.043165   
                                                   RandomForestClassifier       0.071942   
                                                   SVC                          0.215827   
AP_Breast_Ovary                                    BaggingClassifier            0.016760   
                                                   BaselineClassifier           0.402235   
                                                   BernoulliNaiveBayes          0.324022   
                                    

## Save tables to $\LaTeX$

In [16]:
#average and standard error
with open('../mlaut_paper/tables/avg_and_st_error.tex', 'w') as tf:
    tf.write(avg_and_std_error.to_latex())
    
#average rank
with open('../mlaut_paper/tables/avg_rank.tex', 'w') as tf:
    tf.write(avg_rank.to_latex())

#average metrics
with open('../mlaut_paper/tables/avg_metrics.tex', 'w') as tf:
    tf.write(avg_metrics.to_latex())
#Cohen's D
with open('../mlaut_paper/tables/cohens_d.tex', 'w') as tf:
    tf.write(cohens_d.to_latex())
#t-test
with open('../mlaut_paper/tables/t_test.tex', 'w') as tf:
    tf.write(t_test_df.to_latex())
with open('../mlaut_paper/tables/t_test_bonferroni.tex', 'w') as tf:
    tf.write(t_test_bonferroni_df.to_latex())
#sign test
with open('../mlaut_paper/tables/sign_test.tex', 'w') as tf:
    tf.write(sign_test_df.to_latex())
#t-test with Bonferroni correction
with open('../mlaut_paper/tables/t_test_bonferroni.tex', 'w') as tf:
    tf.write(t_test_bonferroni_df.to_latex())
#Wilcoxon
with open('../mlaut_paper/tables/wilxocon_test.tex', 'w') as tf:
    tf.write(wilcoxon_df_multiindex.to_latex())
#Friedman test
with open('../mlaut_paper/tables/friedman_test.tex', 'w') as tf:
    tf.write(friedman_test_df.to_latex())
#Nemeniy test
with open('../mlaut_paper/tables/nemeniy_test.tex', 'w') as tf:
    tf.write(nemeniy_test_df.to_latex())
#Errors per dataset per estimator
with open('../mlaut_paper/tables/errors_per_dataset_per_estimator.tex', 'w') as tf:
    tf.write(errors_per_dataset_per_estimator_df.to_latex())